In [ ]:
# built-in dependencies
import math
import random

# third-party dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Problem 1a

Load the SpotifyFeatures.csv file and report the number of samples (songs) as well as the number of features (song properties) in the dataset.

Hint: you may use the Python module Pandas and its function
read_csv.

In [ ]:
SPOTIFY_CSV: str = "data/SpotifyFeatures.csv"
dataset = pd.read_csv(SPOTIFY_CSV)
print(f"Number of samples:  {dataset.shape[0]}")
print(f"Number of features: {dataset.shape[1]}")

### Problem 1b

You will be working with samples from two genres namely ’Pop’ and ’Classical’.

Retrieve all samples belonging to the two genres and create labels for the samples i.e: ’Pop’ = 1, ’Classical’ = 0.

Report how many samples belongs to the two classes.

Working with all features is not always the best solution since it increases the computational cost and some of them may be useless for the task.

For this dataset you should be able to separate the two classes by using two features, namely ’liveness’ and ’loudness’.

In [ ]:
# remove all samples where 'genre' is not equal to 'Pop' or 'Classical'
dataset = dataset[dataset["genre"].isin(["Pop", "Classical"])]
# transform the feature column 'genre' so that 'Pop' is 1 and 'Classical' is 0
dataset["genre"] = dataset["genre"].map({"Pop": 1, "Classical": 0})
# only include features 'genre', 'liveness' and 'loudness'
dataset = dataset.loc[:, ["genre", "liveness", "loudness"]]
# the modified and filtered dataset is now stored in the dataset variable

# extract and report how many samples belongs to the two classes
print(f"Number of pop genre samples:       {dataset[dataset['genre'] == 1].shape[0]}")
print(f"Number of classical genre samples: {dataset[dataset['genre'] == 0].shape[0]}")

### Problem 1c

From the reduced dataset, make 2 numpy arrays.

The first array will be the matrix with songs along the rows and songs’ features ("liveness" and "loudness") as columns.

This will be the input of our machine learning method.

The second array will the vector with the songs’ genre (labels or target we want to learn).

Create a training and test set by splitting the dataset.

Use an 80% 20% split between the training and test set.

Split the data per class so that you keep the same class distribution in the training and test set.

In [ ]:
# extract only the pop genre
dataset_pop = dataset[dataset["genre"] == 1]
# extract pseudo-randomly 80% to use for training
training_pop = dataset_pop.sample(frac=0.8, random_state=random.randint(1, 100))
# extract pseudo-randomly 20% to use for testing
test_pop = dataset_pop.drop(training_pop.index)

# extract only the classical genre
dataset_classical = dataset[dataset["genre"] == 0]
# extract pseudo-randomly 80% to use for training
training_classical = dataset_classical.sample(frac=0.8, random_state=random.randint(1, 100))
# extract pseudo-randomly 20% to use for testing
test_classical = dataset_classical.drop(training_classical.index)

# merge the two training sets back together, and reset the sample index
train_set = pd.concat([training_pop, training_classical]).reset_index(drop=True)
# merge the two testing sets back together, and reset the sample index
test_set = pd.concat([test_pop, test_classical]).reset_index(drop=True)

# the training set and test set may now be split into two numpy arrays,
# a vector for labels and a matrix for features for each set
# example:
#    labels =  np.array(<train/test>_set["genre"])
#    features =  np.array(<train/test>_set.loc[:, ["liveness", "loudness"]]))

print(f"Number of samples in data set:        {dataset.shape[0]}")
print(f"80% of data set:                      {int(dataset.shape[0] * 0.8)}")
print(f"20% of data set:                      {int(dataset.shape[0] * 0.2)}")
print(f"Number of samples in training set:    {train_set.shape[0]}")
print(f"Number of samples in test set:        {test_set.shape[0]}")
# time for some extra wide monitor code
print(f"Pop percentage of dataset:            {round(dataset[dataset['genre'] == 1].shape[0] / dataset.shape[0] * 100, 2)}")
print(f"Classical percentage of dataset:      {round(dataset[dataset['genre'] == 0].shape[0] / dataset.shape[0] * 100, 2)}")
print(f"Pop percentage of training set:       {round(train_set[train_set['genre'] == 1].shape[0] / train_set.shape[0] * 100, 2)}")
print(f"Classical percentage of training set: {round(train_set[train_set['genre'] == 0].shape[0] / train_set.shape[0] * 100, 2)}")
print(f"Pop percentage of testing set:        {round(test_set[test_set['genre'] == 1].shape[0] / test_set.shape[0] * 100, 2)}")
print(f"Classical percentage of testing set:  {round(test_set[test_set['genre'] == 0].shape[0] / test_set.shape[0] * 100, 2)}")

### Problem 1d - Bonus

Plot the samples on the liveness vs loudness plane, with a different color for each class.

From the plot, will the classification be an easy task? Why?

In [ ]:
size: float = 2
transparency: float = 1

# plot the liveness vs loudness plane with classical on top
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.scatter(
    dataset_classical["liveness"],
    dataset_classical["loudness"],
    color="blue",
    label="classical",
    alpha=transparency,
    s=size
)
plt.scatter(
    dataset_pop["liveness"],
    dataset_pop["loudness"],
    color="red",
    label="pop",
    alpha=transparency,
    s=size
)
plt.xlabel("liveness")
plt.ylabel("loudness")
plt.legend()

# plot the liveness vs loudness plane with pop on top
plt.subplot(1, 2, 2)
plt.scatter(
    dataset_pop["liveness"],
    dataset_pop["loudness"],
    color="red",
    label="pop",
    alpha=transparency,
    s=size
)
plt.scatter(
    dataset_classical["liveness"],
    dataset_classical["loudness"],
    color="blue",
    label="classical",
    alpha=transparency,
    s=size
)
plt.xlabel("liveness")
plt.ylabel("loudness")
plt.legend()

# render the two plots
plt.show()

### Problem 2a

Implement your own logistic discrimination classifier and use the training data to train the classifier.

You should use stochastic gradient descent and implement it in Python.

Plot the training error as a function of epochs, and report the accuracy on the training set.

Try different learning rates for the gradient descent and explain what you observe for these different values.

Optional, it may help the learning process if the data is shuffled (songs are fed to the classifier in random order).

In [ ]:
def sigmoid(z: float) -> float:
    """transforms the prediction to a probability

    Args:
        z (float): y hat or linear prediction

    Returns:
        float: log odds
    """
    return 1/(1 + pow(math.e, -z))

def linear_prediction(X: np.array, w: np.array, b: float) -> np.array:
    """element-wise linear prediction

    Args:
        X (np.array): feature vector
        w (np.array): weights
        b (float): bias

    Returns:
        np.array: prediction or y
    """
    return np.dot(X, w) + b

def binary_cross_entropy(y: float, y_hat: float) -> float:
    """element-wise binary cross entropy function

    note that y_hat is clipped to prevent log(0) and log(1)

    Args:
        y (float): actual label
        y_hat (float): predicted label

    Returns:
        float: loss for one element
    """
    epsilon = 1e-15
    y_hat = np.clip(y_hat, epsilon, 1 - epsilon)
    return -1 * (y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))

def weight_derivative(x, y, y_hat) -> float:
    """derived binary cross entropy function with respect to w(eights)

    Args:
        x (_type_): features or input, should usually be transposed
        y (_type_): true label
        y_hat (_type_): predicted label

    Returns:
        float: gradient descent of w(eights)
    """
    return np.dot(x, y_hat - y)

def bias_derivative(y, y_hat) -> float:
    """derived binary cross entropy function with respect to b(ias)

    Args:
        y (_type_): true label
        y_hat (_type_): predicted label

    Returns:
        float: gradient descent of b(ias)
    """
    return y_hat - y

def overall_accuracy(x: np.array, y: np.array, w: np.array, b: float, lim: float) -> float:
    """calculates general accuracy for classification

    Args:
        x (np.array): features
        y (np.array): true labels
        w (np.array): weights
        b (float): bias
        lim (float): threshold for accepting a probability as 1

    Returns:
        float: overall accuracy in the range 0 to 1
    """
    s: int = 0
    n: int = x.shape[0]
    for i in range(0, n - 1):
        y_hat = sigmoid(linear_prediction(x[i], w, b))
        if (1 if y_hat >= lim else 0) == y[i]:
            s += 1
    return s / n

In [ ]:
n: int = train_set.shape[0]

# initialize bias and weights as zero
bias: float = 0
weights = np.zeros(2)

# set number of training iterations and hyperparameters
epochs: list[int] = [i for i in range(1, 101)]
learning_rate: float = 0.001

# setup variables for training error as a function of epochs
loss: int = 0
error: list[float] = []

# iterate over every epoch
for epoch in epochs:
    # set total loss to zero every epoch
    loss = 0
    # shuffle the training set pseudo-randomly
    set = train_set.sample(frac=1, random_state=random.randint(1, 100))
    # split the shuffled training set into y and x
    labels = np.array(set["genre"])
    features = np.array(set.loc[:, ["liveness", "loudness"]])
    # iterate over every sample in dataset
    for i in range(0, n):
        # calculate linear prediction and apply the sigmoid
        predicted = sigmoid(linear_prediction(features[i], weights, bias))
        # accumulate loss
        loss += binary_cross_entropy(labels[i], predicted)
        # calculate gradient descent for the weights
        weight_gradient = weight_derivative(features[i].T, labels[i], predicted)
        # calculate gradient descent for the bias
        bias_gradient = bias_derivative(labels[i], predicted)
        # update weights and bias
        weights = weights - learning_rate * weight_gradient
        bias = bias - learning_rate * bias_gradient
    # calculate error as the mean loss
    error.append(loss / n)

# calculate the overall accuracy
accuracy: float = overall_accuracy(
    x=np.array(train_set.loc[:, ["liveness", "loudness"]]),
    y=np.array(train_set["genre"]),
    w=weights,
    b=bias,
    lim=0.5
)

# report the final characteristics
print(f"Accuracy: {accuracy}")
print(f"Weights:  {weights}")
print(f"Bias:     {bias}")
print(f"Error:    {error[-1]}")

# plot the error for each epoch to get a loss graph
plt.figure(figsize=(10, 6))
plt.plot(epochs, error)
plt.xlabel("epochs")
plt.ylabel("error")
plt.show()

### Problem 2b

Test your trained logistic discrimination classifier using the test set.

Report the accuracy on the test set.

Is there a significant difference between the accuracy on the training and test set?

If so what might that indicate.

[Optional] Inform (or brag) about your test accuracy score on Discord.

It is nice to seen how other students perform while working on the assignment

In [ ]:
# calculate the overall accuracy for the training set
training_oa: float = overall_accuracy(
    x=np.array(train_set.loc[:, ["liveness", "loudness"]]),
    y=np.array(train_set["genre"]),
    w=weights,
    b=bias,
    lim=0.5
)
# calculate the overall accuracy for the testing set
testing_oa: float = overall_accuracy(
    x=np.array(test_set.loc[:, ["liveness", "loudness"]]),
    y=np.array(test_set["genre"]),
    w=weights,
    b=bias,
    lim=0.5
)
# calculate the difference in overall accuracy
diff_oa: float = abs(training_oa - testing_oa)

# report the findings
print(f"Training accuracy:      {training_oa}")
print(f"Testing accuracy:       {testing_oa}")
print(f"Difference in accuracy: {diff_oa}")

### Problem 2c - Bonus

Extract the learned parameters from your logistic regression and use them to draw the linear line separating the data on the plot you made in question (1d).

This may help you understand why your classifier is performing well or not.

In [ ]:
# create 100 evenly spaced values from min liveness to max liveness
x_values = np.linspace(min(dataset["liveness"]), max(dataset["liveness"]), 100)
# loudness as a function of liveness
# z = w1 ​* liveness + w2 ​* loudness + b
# =>
# loudness = y_values = -(w0 / w1) * liveness - (bias / w1)
y_values = -(weights[0] / weights[1]) * x_values - (bias / weights[1])

# plot the liveness vs loudness plane, with label boundary
plt.figure(figsize=(10, 6))
plt.scatter(
    dataset_pop["liveness"],
    dataset_pop["loudness"],
    color="red",
    label="pop",
    alpha=transparency,
    s=size
)
plt.scatter(
    dataset_classical["liveness"],
    dataset_classical["loudness"],
    color="blue",
    label="classical",
    alpha=transparency,
    s=size
)
plt.plot(x_values, y_values, color="green", label="boundary")
plt.xlabel("liveness")
plt.ylabel("loudness")
plt.legend()
plt.show()

### Problem 3a

Using the classification results from the test set in problem 2, create a confusion matrix for the classification.

Report the confusion matrix.

In [ ]:
def confusion_matrix(x: np.array, y: np.array, w: np.array, b: float, lim: float) -> float:
    """calculates the confusion matrix

    Args:
        x (np.array): features
        y (np.array): labels
        w (np.array): weights
        b (float): bias
        lim (float): threshold for accepting a probability as 1

    Returns:
        float: a 2 by 2 matrix where [["TP", "FN"], ["FP", "TN"]]
    """
    TP = 0; FP = 0; TN = 0; FN = 0
    y_hat = np.zeros(len(y))
    # for each input value
    for i in range(0, len(x)):
        # calculate the predicted label
        probability = sigmoid(linear_prediction(x[i], w, b))
        y_hat[i] = 1 if probability >= lim else 0
        # actual label is positive and predicted label is positive
        if   y[i] == 1 and y_hat[i] == 1:
            TP += 1
        # actual label is negative and predicted label is positive
        elif y[i] == 0 and y_hat[i] == 1:
            FP += 1
        # actual label is negative and predicted label is negative
        elif y[i] == 0 and y_hat[i] == 0:
            TN += 1
        # actual label is positive and predicted label is negative
        elif y[i] == 1 and y_hat[i] == 0:
            FN += 1
    # build and return the confusion matrix
    return np.array([
        [TP, FN],
        [FP, TN]
    ])

# calculate the confusion matrix
cm = confusion_matrix(
    x=np.array(test_set.loc[:, ["liveness", "loudness"]]),
    y=np.array(test_set["genre"]),
    w=weights,
    b=bias,
    lim=0.5
)

# calculate and report different evaluation metrics
print(f"Accuracy:  {round((cm[0, 0]+ cm[1, 1]) / (cm[0, 0] + cm[1, 1] + cm[1, 0] + cm[0, 1]), 2)}")
print(f"F1:        {round((2 * cm[0, 0]) / (2 * cm[0, 0] + cm[1, 0] + cm[0, 1]), 2)}")
print(f"Precision: {round((cm[0, 0]) / (cm[0, 0] + cm[1, 0]), 2)}")
print(f"Recall:    {round((cm[0, 0]) / (cm[0, 0] + cm[0, 1]), 2)}")

# draw a confusion matrix for better visual aid, took inspiration from
# this link https://vitalflux.com/python-draw-confusion-matrix-matplotlib/
fig, ax = plt.subplots(figsize=(6, 6))
ax.matshow(cm, cmap=plt.cm.Blues, alpha=0.3)
tick_marks = np.array([["TP", "FN"], ["FP", "TN"]])
for i in range(len(cm)):
    for j in range(len(cm[0])):
        ax.text(x=j, y=i, s=f"{tick_marks[i, j]}: {cm[i, j]}", va="center", ha="center")
plt.ylabel("actual label")
plt.xlabel("predicted label")
plt.show()


### Problem 3b

You should now have two evaluation metrics for the performance of the classifier on the test set; accuracy and the confusion matrix.

What information does the confusion matrix give you that the accuracy score does not?

NO CODE

### Problem 3c - Bonus

Which songs are difficult to classify?

Could you suggest some Classical songs that a Pop fan would like? (a good song could influence positively the mood of the evaluator)

NO CODE